# Cleaning the 2021 NYC Parking Violation Dataset

This is the beginning of my very first independent project after completing IBM's Professional Data Analyst Specialization at the end of March, 2022.

<p>Roughly 14.7 million violations were issued by NYC in fiscal year 2021 (July 2020 - June 2021). Our final cleaned dataset is composed of about 6.6 million rows.
<br>Unfortunately, many data entries containing null or 'intranslatable' values (i.e. violation time of '45:21 PM') were removed, primarily due to the limitations of my PC.
<br>Efforts were made to preserve as much data as possible after initial nan drop by replacing with median/mode/manually input spell-check key:value pairs.</p>

The main dataset was obtained from <a href="https://data.cityofnewyork.us/City-Government/Parking-Violations-Issued-Fiscal-Year-2021/kvfd-bves">NYC OpenData</a> using the Socrata OpenData API.

Violation and vehicle codes were converted to full text descriptions using information 'scraped' from official nyc.gov documentation. 
<br>Identical values written in numerous different ways were standardized when able. 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from datetime import datetime as dt
from sodapy import Socrata

#### Importing dataset using Socrata API:

In [3]:
# Ordered by date ascending 2021 fiscal year begins on row 254353 and ends on row 14955034
data_url = 'data.cityofnewyork.us'
data_set = 'kvfd-bves'
app_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
client = Socrata(data_url, app_token)
client.timeout = 90
start = 254353
chunk_size = 50000
end = 14955034
results = []
while True:
     # All rows beginning at 'start':
     results.extend( client.get(data_set, 
                     select="""
                     summons_number, registration_state, plate_type, issue_date, violation_code, vehicle_body_type, 
                     vehicle_make, violation_precinct, issuer_code, violation_time, violation_county, house_number, 
                     street_name, vehicle_color, vehicle_year""",
                     order='issue_date',
                     offset=start,
                     limit=chunk_size))
     start = start + chunk_size
     if (start > end):
        break
df = pd.DataFrame.from_records(results)

#### Cleaning/Pruning:

In [4]:
df

,summons_number,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,violation_precinct,issuer_code,violation_time,violation_county,house_number,street_name,vehicle_color,vehicle_year
0,1471514882,NY,PAS,2020-07-01,98,SUBN,FORD,102,949909,1255P,Q,81-02,ROCKAWAY BLVD,WH,1998
1,1471514894,NY,PAS,2020-07-01,98,SUBN,NISSA,102,949969,0143P,Q,87-34,132 ST,RED,2018
2,1471514900,NY,PAS,2020-07-01,98,SUBN,INFIN,102,949969,0200P,Q,87-45,112 ST,BLACK,2004
3,1473991845,NY,PAS,2020-07-01,48,SUBN,INFIN,40,952829,0442P,BX,418,E 138 ST,BK,2008
4,1473991857,NY,PAS,2020-07-01,48,SUBN,HYUN,40,952829,0444P,BX,416,E 138 ST,WH,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14700677,1479707030,NY,PAS,2021-12-31,21,SUBN,NaN,48,688077,1009A,BX,2353,SOUTHERN BLVD,WHITE,2002
14700678,1481537994,NY,PAS,2021-12-31,21,SDN,HONDA,43,584562,0838A,BX,1720,METROPOLITAN AVENUE,BLACK,2012
14700679,1481741330,PA,PAS,2021-12-31,21,SDN,VOLKS,62,592234,1201P,K,1715,78TH STREET,WHT,0
14700680,1481741342,NY,PAS,2021-12-31,21,SDN,NaN,62,592234,1159A,K,1745,78TH STREET,GRY,2017


In [5]:
df.isnull().sum()

summons_number              0
registration_state          0
plate_type                  0
issue_date                  0
violation_code              0
vehicle_body_type       40086
vehicle_make            33301
violation_precinct          0
issuer_code                 0
violation_time            247
violation_county        32059
house_number          5016861
street_name              1303
vehicle_color          763905
vehicle_year                0
dtype: int64

In [6]:
# Remove rows with NaN to protect CPU
df2 = df.dropna()

In [7]:
df2.shape

(9595811, 15)

#### Standardizing Vehicle Year:

In [10]:
pd.options.mode.chained_assignment = None

In [11]:
df2['vehicle_year'] = df2['vehicle_year'].astype('int64')

In [12]:
# Find Median value
df2[(df2['vehicle_year'] > 0) & (df2['vehicle_year'] <= 2022)].vehicle_year.median()

2015.0

In [14]:
# Convert to datetime
df2.loc[:, ['issue_date']] = pd.to_datetime(df['issue_date'])

# Remove rows after 2021
df2 = df2[df2['issue_date'] <= '2021-12-31']

# Removing the 3 million 'year 0' rows to further ease processing and to avoid heavily skewing density:
df2.drop(df2[df2['vehicle_year'] == 0].index, inplace=True)

# Replacing invalid years with median:
df2.loc[df2['vehicle_year'] >= 2023, ['vehicle_year']] = 2015

In [15]:
df2

,summons_number,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,violation_precinct,issuer_code,violation_time,violation_county,house_number,street_name,vehicle_color,vehicle_year
0,1471514882,NY,PAS,2020-07-01,98,SUBN,FORD,102,949909,1255P,Q,81-02,ROCKAWAY BLVD,WH,1998
1,1471514894,NY,PAS,2020-07-01,98,SUBN,NISSA,102,949969,0143P,Q,87-34,132 ST,RED,2018
2,1471514900,NY,PAS,2020-07-01,98,SUBN,INFIN,102,949969,0200P,Q,87-45,112 ST,BLACK,2004
3,1473991845,NY,PAS,2020-07-01,48,SUBN,INFIN,40,952829,0442P,BX,418,E 138 ST,BK,2008
4,1473991857,NY,PAS,2020-07-01,48,SUBN,HYUN,40,952829,0444P,BX,416,E 138 ST,WH,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14700673,1481807882,NY,PAS,2021-12-30,21,SDN,KIA,62,625050,1136A,K,6105,17 AVE,GY,2020
14700674,1475217948,NY,COM,2021-12-30,84,DELV,INTER,0,962891,0813A,NY,529,5TH AVE,WH,2018
14700675,1481886733,NY,PAS,2021-12-30,40,SUBN,BMW,104,943660,0555P,Q,71-34,58RD,BLACK,2020
14700678,1481537994,NY,PAS,2021-12-31,21,SDN,HONDA,43,584562,0838A,BX,1720,METROPOLITAN AVENUE,BLACK,2012


In [17]:
df2.describe(exclude=['datetime64[ns]'])

,summons_number,registration_state,plate_type,violation_code,vehicle_body_type,vehicle_make,violation_precinct,issuer_code,violation_time,violation_county,house_number,street_name,vehicle_color,vehicle_year
count,6581598,6581598,6581598,6581598,6581598,6581598,6581598,6581598,6581598,6581598,6581598,6581598,6581598,6.581598e+06
unique,6581598,58,79,95,642,1366,142,18639,1562,12,42026,23748,696,NaN
top,1471514882,NY,PAS,21,SUBN,FORD,19,362237,0836A,NY,N,Broadway,WH,NaN
freq,1,6550778,4576755,1164056,2715963,798897,326170,11844,23351,2441574,290595,139656,1627996,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.013463e+03
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.841012e+00
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.970000e+03
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.010000e+03
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.015000e+03
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.018000e+03


#### Standardizing Precincts:
Of note, all 4.5 million rows containing Precinct: '0' were dropped together with the na rows.

In [19]:
df2.violation_precinct.value_counts()

19     326170
114    260320
18     208086
13     201620
14     191847
        ...  
808         1
55          1
140         1
824         1
923         1
Name: violation_precinct, Length: 142, dtype: int64

In [21]:
# Convert violation_precinct to int
df2['violation_precinct'] = pd.to_numeric(df2.violation_precinct)

# Remove all rows with precinct > 123 (no such precincts exist in NYC)
df2 = df2[df2['violation_precinct'].between(1, 123, inclusive='both')]

In [22]:
# Plate Types and State Codes using PDF: http://www.nyc.gov/html/dof/html/pdf/faq/stars_codes.pdf
plates = 'C:\\Users\\sarzy\\Documents\\Jupyter-lab\\2022 Parking Violations\\platetypes.xlsx'
states = 'C:\\Users\\sarzy\\Documents\\Jupyter-lab\\2022 Parking Violations\\territorycodes.xlsx'
plate_df = pd.read_excel(plates, index_col=0)
state_df = pd.read_excel(states, index_col=0)

In [23]:
dict1 = plate_df.to_dict()
dict2 = state_df.to_dict()
plate_dict = dict1['plate_types']
state_dict = dict2['territory']

In [24]:
df3 = df2.replace({'registration_state': state_dict})
df3.replace({'plate_type': plate_dict}, inplace = True)

In [25]:
# Violation codes and fines from NYC Open Data
vios = 'C:\\Users\\sarzy\\Documents\\Jupyter-lab\\2022 Parking Violations\\ParkingViolationCodes_January2020.xlsx'
vio_df = pd.read_excel(vios)

In [26]:
# Set as dictionary
viocodes = vio_df[['violation_code', 'violation_description']].set_index('violation_code').to_dict()
viofine = vio_df[['violation_description', 'fine_amount', 'manhattan_fine']].set_index('violation_description').to_dict()
viocodes_dict = viocodes['violation_description']

# Fines from all boroughs but Manhattan
viofine_dict = viofine['fine_amount']

# Manhattan fines
viofine_nyc_dict = viofine['manhattan_fine']

In [27]:
df3['violation_code'] = pd.to_numeric(df3.violation_code)

# Replace numerical code with full description of violation
df3.replace({'violation_code':viocodes_dict}, inplace=True)

# Create copy of violations column to use as keys for dict replace
df3['violation'] = df3['violation_code']

# Replace all rows in Manhattan with Manhattan fine value
df3.loc[df3['violation_county'] == 'NY', 'violation_code'] = df3.replace({'violation_code':viofine_nyc_dict})
        
# Since Manhattan fine values are not keys in dict, can use replace on whole column
df3.replace({'violation_code':viofine_dict}, inplace=True)

df3.rename(columns={'violation_code':'violation_fine'}, inplace=True)

### Cleaning the Violation Time:

In [28]:
df3.violation_time.head()

0    1255P
1    0143P
2    0200P
3    0442P
4    0444P
Name: violation_time, dtype: object

In [30]:
import re
# Convert '0000P' to '0000 PM'
times = df3.violation_time.tolist()
time_formatted = []
for time in times:
    # If time string ends in P or A, replace with ' PM/AM'
    if re.search('.+P', time):
        time_formatted.append(time.replace('P', ' PM'))
    elif re.search('.+A', time):
        time_formatted.append(time.replace('A', ' AM'))
    # Any culprits without proper P or A format are unaltered and appended to list to preserve index
    else:
        time_formatted.append(time)

In [31]:
# Finding the culprits:
culprits = []
for suspect in time_formatted:
    if not re.search(('.+A'), suspect):
        if not re.search(('.+P'), suspect):
            culprits.append(suspect)

In [32]:
culprits

['0854',
 '0625',
 '0420',
 '0450',
 '0450',
 '1235',
 '1116',
 '0910',
 '0207',
 '1052',
 '0609',
 '1234']

In [34]:
# We are assuming a 7 AM - 7 PM workday for reformatting
for culprit in culprits:
    if culprit[0:2] in ['08','11','09','10']:
        time_formatted[time_formatted.index(culprit)] = (culprit + ' ' + 'AM')
    else:
        time_formatted[time_formatted.index(culprit)] = (culprit + ' ' + 'PM')

# Another 2 anomalies found in below cell:
time_formatted[time_formatted.index('110  AM')] = '1100 AM'
time_formatted[time_formatted.index('040/ PM')] = '0400 PM'

In [35]:
# Checking for potential formatting issues:
wrong_length = []
for time in time_formatted:
    if len(time) != 7:
        wrong_length.append(time)
print(wrong_length)
print((len(df3.violation_time)) == (len(time_formatted)))

[]
True


In [36]:
# Checking for minute values > 59
wrong_minutes = []
for time in time_formatted:
    if time[2:4] >= '60':
        wrong_minutes.append(time)
print(wrong_minutes)

[]


#### Standardizing times to hours in 12H format, isolating values with H > 24, adding colon for style:

In [37]:
# Defining hour dictionary and empty lists
time_dict = {'13':'01:', '14':'02:', '15':'03:', '16':'04:', '17':'05:', '18':'06:','19':'07:','20':'08:','21':'09:','22':'10:','23':'11:'}
time_list = []
times_over_24 = []

for time in time_formatted:
    if '00' <= time[0:2] <= '24':                                    # Time values within 00H - 24H range
        if '01' <= time[0:2] <= '12':                                # Time values that have hours in 12H format (01 - 12)
            time_list.append(time[0:2] + ':00' + time[4:])
        elif '00' == time[0:2] or '24' == time[0:2]:                 # Midnight hours (00/24)
            if re.search('.+AM', time):
                time_list.append('12:00' + time[4:])
            else:                                                    # PM values
                time_list.append('12:00 ' + 'AM')
        elif '13' <= time[0:2] <= '23':                              # Our data does not contain AM times with hour >12
            for k,v in time_dict.items():                            # Iterate through keys, values in hour dictionary. colon already in values.
                if k in time[0:2]:
                    time_list.append(v + '00' + time[4:])                
    else:                                                            # All times with hour > 24 (only PM strings exist in our data)
        time_list.append(time[0:2] + ':00' + time[4:])
        times_over_24.append(time[0:2] + ':00' + time[4:])

In [38]:
# Save indices of times over 24 for drop
bad_time_index = []
for time in times_over_24:
    bad_time_index.append(time_list.index(time))
times_over_24

['48:00 PM',
 '43:00 PM',
 '38:00 PM',
 '56:00 PM',
 '28:00 PM',
 '54:00 PM',
 '56:00 PM',
 '71:00 PM',
 '69:00 PM',
 '58:00 PM',
 '76:00 PM',
 '83:00 PM']

In [39]:
# Replace time values with reformatted list
df3['violation_time'] = time_list

# Since index was preserved for times > 24H, we can drop according to index list
df4 = df3.drop(df3.index[bad_time_index])

In [40]:
df4.describe(exclude=['datetime64[ns]', 'int64', 'float64'])

,summons_number,registration_state,plate_type,vehicle_body_type,vehicle_make,issuer_code,violation_time,violation_county,house_number,street_name,vehicle_color,violation
count,6578807,6578807,6578807,6578807,6578807,6578807,6578807,6578807,6578807,6578807,6578807,6578807
unique,6578807,58,79,636,1357,18477,25,11,42015,23652,691,95
top,1471514882,NEW YORK,Passenger,SUBN,FORD,362237,08:00 AM,NY,N,Broadway,WH,NO PARKING-STREET CLEANING
freq,1,6548271,4575218,2715265,798583,11844,767168,2439911,290595,139656,1627379,1163762


### For future revisions where .dropna() is not used - Replace body type na with mode per vehicle make:

In [41]:
# Upper case
df4['vehicle_make'] = df4.vehicle_make.str.upper()
df4['vehicle_body_type'] = df4.vehicle_body_type.str.upper()

In [42]:
df_cars = df4[['vehicle_make', 'vehicle_body_type']]

# Find most frequent body types by vehicle make
df_car_grp = df_cars.groupby(['vehicle_make', 'vehicle_body_type'], as_index = False).size()
df_car_grp.sort_values(by=['size'], ascending = False, inplace = True)

In [43]:
df_car_grp.head()

,vehicle_make,vehicle_body_type,size
1002,FORD,VAN,414376
1482,HONDA,SUBN,351734
1368,HONDA,4DSD,329572
3644,TOYOT,SUBN,328325
3545,TOYOT,4DSD,290032


In [44]:
# Drop duplicate vehicle makes to keep only mode body type per make and create dictionary
df_car_grp.drop_duplicates(subset = 'vehicle_make', inplace = True)
car_dict = df_car_grp[['vehicle_make', 'vehicle_body_type']].set_index('vehicle_make').head(50).to_dict()
car_dict = car_dict['vehicle_body_type']

In [45]:
# Create df with remaining NaN rows
narows = df4[df4.isna().any(axis=1)]

# Replace NaN values with corresponding vechicle make's mode body type 
narows['vehicle_type'] = narows['vehicle_make']
narows.replace({'vehicle_type':car_dict}, inplace=True)
narows['vehicle_body_type'] = narows['vehicle_type']
narows.drop(columns = 'vehicle_type', inplace = True)

In [46]:
# Replace main df NaN rows by matching with temp df index
df4.loc[narows.index, :] = narows[:]

### Misc Data Standardization

In [47]:
# Upper-casing
df4 = df4.applymap(lambda x: x.upper() if type(x) == str else x)

In [48]:
# Precinct number as int
df4['violation_precinct'] = df4.violation_precinct.astype('int64')

In [49]:
# Create month-year column
import datetime
df4['month_year'] = pd.DatetimeIndex(df4['issue_date']).strftime('%b %Y').str.upper()

### Standardizing Location Info

In [50]:
# Standardize violation county as borough name
county_dict = {'NY': 'MANHATTAN', 'K': 'BROOKLYN', 'Q': 'QUEENS', 'BX': 'BRONX', 'R': 'STATEN ISLAND', 
               'KINGS':'BROOKLYN', 'QNS': 'QUEENS', 'F': 'QUEENS', 'RICH':'STATEN ISLAND'} #Based off address, F stands for Flushing, Queens
df4.replace({'violation_county':county_dict}, inplace = True) 

In [51]:
# Merge house_number and street_name into address
df4['address'] = df4[['house_number', 'street_name', 'violation_county']].agg(' '.join, axis=1)

### Standardizing Vehicle Color

In [52]:
# https://data.ny.gov/api/assets/83055271-29A6-4ED4-9374-E159F30DB5AE contains NYC color codes
color_counts = df4.groupby('vehicle_color', as_index = False).size()
color_counts.sort_values(by=['size'], ascending = False, inplace = True)
color_counts.head()

,vehicle_color,size
621,WH,1627379
341,GY,1443955
59,BK,1255318
78,BL,507389
640,WHITE,340011


In [54]:
# Color code dict derived from https://data.ny.gov/api/assets/83055271-29A6-4ED4-9374-E159F30DB5AE and manual interpretation of frequent misspellings 
color_dict = {"BK":"BLACK","BL":"BLUE", "BR":"BROWN","GL":"YELLOW","GY":"GRAY","MR":"RED","OR":"ORANGE","PK":"PINK","PR":"PURPLE","RD":"RED","TN":"BROWN","WH":"WHITE","YW":"YELLOW","NOCL":"NO COLOR",
             "GRY":"GRAY","GRN":"GREEN","BURG":"RED","SIL":"GRAY","WHT":"WHITE","BLK":"BLACK", "GOLD":"YELLOW","GLD":"YELLOW","SILVER":"GRAY","GREY":"GRAY","BLU":"BLUE","GRY":"GRAY", "GR":"GRAY",
             "SILVE":"GRAY","MAROON":"RED","PURPL":"PURPLE","YELLO":"YELLOW","WHI":"WHITE","WT":"WHITE","ORANG":"ORANGE","DKG":"GRAY","GY":"GRAY","GY/":"GRAY","TAN":"BROWN","W":"WHITE","BRN":"BROWN",
              "BRO":"BROWN","SILV":"GRAY","SLV":"GRAY","WH/":"WHITE","SL":"GRAY","NO":"NO COLOR","OTHER":"NO COLOR","DKB":"BLUE","LTB":"BLUE","NOC":"NO COLOR","WHB":"WHITE","B":"BLACK","LTG":"GRAY",
              "LT/":"GRAY","DK/":"BLACK","GYB":"GRAY","RD/":"RED","BK/":"BLACK","RDW":"RED","WHG":"WHITE","GN":"GREEN","BLG":"BLACK","GRW":"GRAY","BL/":"BLUE","G":"GRAY","BLW":"BLACK","LTT":"BROWN",
              "BN":"BROWN","BKG":"BLACK","LAVEN":"PURPLE","RDG":"RED","MAROO":"RED","WHTE":"WHITE","LTGY":"GRAY","OTH":"NO COLOR","YELL":"YELLOW","LT/GRY":"GRAY","YEL":"YELLOW","BW":"BROWN","BRW":"BROWN",
             "BEIGE":"BROWN","BKW":"BLACK","GD":"YELLOW","DKR":"RED","LT/GY":"GRAY",'GR/':"GRAY","BLCK":"BLACK","BLB":"BLUE","SLVR":"GRAY","MAR":"RED","SV":"GRAY","BRWN":"BROWN","DK/GY":"GRAY","R":"RED",
             "BLA":"BLACK","BLAK":"BLACK","WHIT":"WHITE","SILVR":"GRAY","DKP":"PURPLE","GRB":"GRAY","DKGY":"GREEN","GRG":"GREEN","DKGY":"GRAY","GRG":"GRAY","BUR":"RED","GYG":"GRAY","DKM":"RED","BURGU":"RED",
             "BROW":"BROWN","RDT":"RED","WHO":"WHITE","WTE":"WHITE","SIV":"GRAY","BLE":"BLUE","BLAC":"BLACK","PURP":"PURPLE","BG":"BROWN","ORG":"ORANGE","GRE":"GREEN","TNG":"BROWN","DKBL":"BLUE","PUR":"PURPLE",
             "S":"GRAY","BLN":"BLUE","GRA":"GRAY","Y":"YELLOW","GRT":"GRAY","BIEGE":"BROWN","WHE":"WHITE","BLT":"BLUE","BKT":"BLACK","GREN":"GREEN","DK/RD":"RED","ORA":"ORANGE","TN/":"BROWN","ORAN":"ORANGE",
             "YE":"YELLOW","LTBL":"BLUE","BGE":"BROWN","BRON":"BROWN","YLW":"YELLOW","TNR":"BROWN","LTGR":"GRAY","LTP":"PURPLE","BWN":"BROWN","DKRD":"RED","DK/BL":"BLUE","BL/GY":"BLUE","LT/BL":"BLUE",
              "YL":"YELLOW", "MULTI":"NO COLOR","RUST":"BROWN","PRB":"PURPLE","BRT":"BROWN","B L":"BLACK","GYT":"GRAY","LT/GR":"GRAY","MARON":"RED","GLB":"YELLOW","DK/GR":"GRAY","LT/TN":"BROWN","BU":"BLUE",
             "BURGA":"RED","BKL":"BLACK","SLVE":"GRAY","NO CL":"NO COLOR","BLWH":"BLACK","MRG":"RED","DKGR":"GRAY","TEAL":"GREEN","BY":"BLACK","GEY":"GRAY","BLRD":"BLUE","SLR":"GRAY","ORN":"ORANGE","GT":"GRAY",
             "O":"ORANGE","GAY":"GRAY","NLK":"BLACK","GOL":"YELLOW","GARY":"GRAY","NAVY":"BLUE","WJ":"WHITE","RE":"RED","BKJ":"BLACK","BEI":"BROWN","MA":"RED","WHITW":"WHITE","BLC":"BLACK","SLVER":"GRAY",
             "SLIVE":"GRAY","BURGE":"RED"}
df4.replace({'vehicle_color':color_dict}, inplace = True) 

### Standardizing Vehicle Body Type

In [55]:
# https://data.ny.gov/api/assets/83055271-29A6-4ED4-9374-E159F30DB5AE
body_counts = df4.groupby('vehicle_body_type', as_index = False).size()
body_counts.sort_values(by=['size'], ascending = False, inplace = True)
body_counts.head()

,vehicle_body_type,size
537,SUBN,2715265
1,4DSD,1801579
609,VAN,1131337
208,DELV,244794
456,PICK,198803


In [56]:
body_dict = {"FIRE":"FIRE TRUCK","CONV":"CONVERTIBLE","SEDN":"SEDAN","SUBN":"SUV","4DSD":"FOUR-DOOR SEDAN","2DSD":"TWO-DOOR SEDAN","H/WH":"HOUSE ON WHEELS","ATV":"ALL TERRAIN VEHICLE",
             "MCY":"MOTORCYCLE","H/IN":"HEARSE","LOCO":"LOCOMOTIVE","CUST":"CUSTOM","RPLC":"REPLICA","AMBU":"AMBULANCE","P/SH":"POWER SHOVEL","RBM":"ROAD BUILDING MACHINE","R/RD":"ROAD ROLLER",
             "RD/S":"ROAD SWEEPER","S/SP":"SAND SPRAYER","SN/P":"SNOW PLOW","TRAV":"SNOW TRAVELER","MOBL":"SNOWMOBILE","TR/E":"TRACTION ENGINE","T/CR":"TRACTOR CRANE","TR/C":"TRUCK CRANE",
             "SWT":"TRUCK W/SMALL WHEELS","W/DR":"WELL DRILLER","W/SR":"WELL SERVICING RIG","FPM":"FEED PROCESSING MACHINE","MCC":"MOBILE CAR CRUSHER","EMVR":"EARTH MOVER","TRAC":"TRACTOR",
             "N/A":"NOT APPLICABLE","DELV":"DELIVERY TRUCK","DUMP":"DUMP TRUCK","FLAT":"FLAT BED TRUCK","PICK":"PICK-UP TRUCK","STAK":"STAKE TRUCK","TANK":"TANK TRUCK","REFG":"SEMI-TRAILER",
             "TOW":"TOW TRUCK","UTIL":"UTILITY","POLE":"POLE TRAILER","BOAT":"BOAT","H/TR":"HOUSE TRAILER","SEMI":"SEMI-TRAILER","TRLR":"SEMI-TRAILER","LTRL":"LIGHT TRAILER",
             "LSVT":"LOW SPEED VEHICLE - TRUCK","BUS":"BUS(OMNIBUS)","LIM":"LIMOUSINE(OMNIBUS)","HRSE":"HEARSE(AMBULANCE)","TAXI":"TAXI","DCOM":"DISABLED COMMERICAL","CMIX":"CEMENT MIXER",
             "MOPD":"MOPED","MFH":"MANUFACTURED HOME","SNOW":"SNOWMOBILE","LSV":"LOW SPEED VEHICLE","SDN":"SEDAN","P-U":"PICK-UP TRUCK","FOUR":"FOUR-DOOR SEDAN","TRUC":"TRUCK", "TR":"TRUCK",
            "TT":"SEMI-TRAILER","TRAI":"SEMI-TRAILER","MOT":"MOTORCYCLE","MC":"MOTORCYCLE","TRC":"SEMI-TRAILER","TR":"SEMI-TRAILER","REFR":"SEMI-TRAILER","MOPE":"MOPED","TRC":"TRUCK CRANE","RF":"SEMI-TRAILER",
            "TLR":"SEMI-TRAILER","TRL":"SEMI-TRAILER","TK":"TRUCK","TRK":"TRUCK","REFRIGERATOR TRAILER":"SEMI-TRAILER", "TL":"SEMI-TRAILER","MOTO":"MOTORCYCLE","HWH":"HOUSE ON WHEELS",
            "CM":"CEMENT MIXER","VAN TRUCK":"VAN","SUBURBAN":"SUV","RV":"HOUSE ON WHEELS", "HOUSE TRAILER":"HOUSE ON WHEELS","JEEP":"SUV","FREI":"SEMI-TRAILER"}
df4.replace({'vehicle_body_type':body_dict}, inplace = True) 

In [57]:
df4.head()

,summons_number,registration_state,plate_type,issue_date,violation_fine,vehicle_body_type,vehicle_make,violation_precinct,issuer_code,violation_time,violation_county,house_number,street_name,vehicle_color,vehicle_year,violation,month_year,address
0,1471514882,NEW YORK,PASSENGER,2020-07-01,95,SUV,FORD,102,949909,12:00 PM,QUEENS,81-02,ROCKAWAY BLVD,WHITE,1998,OBSTRUCTING DRIVEWAY,JUL 2020,81-02 ROCKAWAY BLVD QUEENS
1,1471514894,NEW YORK,PASSENGER,2020-07-01,95,SUV,NISSA,102,949969,01:00 PM,QUEENS,87-34,132 ST,RED,2018,OBSTRUCTING DRIVEWAY,JUL 2020,87-34 132 ST QUEENS
2,1471514900,NEW YORK,PASSENGER,2020-07-01,95,SUV,INFIN,102,949969,02:00 PM,QUEENS,87-45,112 ST,BLACK,2004,OBSTRUCTING DRIVEWAY,JUL 2020,87-45 112 ST QUEENS
3,1473991845,NEW YORK,PASSENGER,2020-07-01,115,SUV,INFIN,40,952829,04:00 PM,BRONX,418,E 138 ST,BLACK,2008,BIKE LANE,JUL 2020,418 E 138 ST BRONX
4,1473991857,NEW YORK,PASSENGER,2020-07-01,115,SUV,HYUN,40,952829,04:00 PM,BRONX,416,E 138 ST,WHITE,2012,BIKE LANE,JUL 2020,416 E 138 ST BRONX


In [59]:
# Removing values after June 2021
df4 = df4.loc[(df4['month_year'] != 'JUL 2021') & (df4['month_year'] != 'AUG 2021') & (df4['month_year'] != 'SEP 2021') & (df4['month_year'] != 'OCT 2021') & 
              (df4['month_year'] != 'NOV 2021') & (df4['month_year'] != 'DEC 2021')]

In [60]:
# Creating vehicle age column: 2022 models were released in 2021.
df4['vehicle_age'] = 2022 - df4['vehicle_year']

In [61]:
df4.to_csv('C:\\Users\\sarzy\\Documents\\Jupyter-lab\\2022 Parking Violations\\2021_parking_violations_cleaned_all_final.csv', index = False)

In [62]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6578587 entries, 0 to 14700213
Data columns (total 19 columns):
 #   Column              Dtype         
---  ------              -----         
 0   summons_number      object        
 1   registration_state  object        
 2   plate_type          object        
 3   issue_date          datetime64[ns]
 4   violation_fine      int64         
 5   vehicle_body_type   object        
 6   vehicle_make        object        
 7   violation_precinct  int64         
 8   issuer_code         object        
 9   violation_time      object        
 10  violation_county    object        
 11  house_number        object        
 12  street_name         object        
 13  vehicle_color       object        
 14  vehicle_year        int64         
 15  violation           object        
 16  month_year          object        
 17  address             object        
 18  vehicle_age         int64         
dtypes: datetime64[ns](1), int64(4), object(14